# Наука о данных в облаке: подход "Azure ML SDK"

## Введение

В этом ноутбуке мы изучим, как использовать Azure ML SDK для обучения, развертывания и использования модели через Azure ML.

Предварительные требования:
1. Вы создали рабочую область Azure ML.
2. Вы загрузили [датасет о сердечной недостаточности](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) в Azure ML.
3. Вы загрузили этот ноутбук в Azure ML Studio.

Следующие шаги:

1. Создать эксперимент в существующей рабочей области.
2. Создать вычислительный кластер.
3. Загрузить датасет.
4. Настроить AutoML с использованием AutoMLConfig.
5. Запустить эксперимент AutoML.
6. Изучить результаты и получить лучшую модель.
7. Зарегистрировать лучшую модель.
8. Развернуть лучшую модель.
9. Использовать конечную точку.

## Импорты, специфичные для Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Инициализация рабочей области
Инициализируйте объект рабочей области из сохраненной конфигурации. Убедитесь, что файл конфигурации находится по пути .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Создание эксперимента в Azure ML

Давайте создадим эксперимент с именем 'aml-experiment' в рабочей области, которую мы только что инициализировали.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Создание вычислительного кластера
Вам нужно будет создать [вычислительную цель](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) для запуска AutoML.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Данные
Убедитесь, что вы загрузили набор данных в Azure ML и что ключ имеет то же имя, что и набор данных.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## Конфигурация AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Автозапуск ML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Развертывание лучшей модели

Запустите следующий код, чтобы развернуть лучшую модель. Вы можете увидеть состояние развертывания в портале Azure ML. Этот шаг может занять несколько минут.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Использование конечной точки
Вы можете добавить данные в следующий пример ввода.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
